# Markov Chains: Irreducibility and Ergodicity

In addition to what’s in Anaconda, this lecture will need the following libraries:

```ipython3
!pip install quantecon
!pip install graphviz
```


# graphviz

If you are running this lecture locally it requires [graphviz](https://www.graphviz.org)
to be installed on your computer. Installation instructions for graphviz can be found
[here](https://www.graphviz.org/download/)

## Overview

This lecture continues the journey in Markov chain.

Specifically, we will introduce irreducibility and ergodicity, and how they connect to stationarity.

Irreducibility is a concept that describes the ability of a Markov chain to move between any two states in the system.

Ergodicity is a sample path property that describes the behavior of the system over long periods of time.

The concepts of irreducibility and ergodicity are closely related to the idea of stationarity.

An irreducible Markov chain guarantees the existence of a unique stationary distribution, while an ergodic Markov chain ensures that the system eventually reaches its stationary distribution, regardless of its initial state.

Together, these concepts provide a foundation for understanding the long-term behavior of Markov chains.

Let’s start with some standard imports:

```ipython3
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  # set default figure size
import quantecon as qe
import numpy as np
from graphviz import Digraph
import networkx as nx
from matplotlib import cm
import matplotlib as mpl
```


## Irreducibility

Irreducibility is a central concept of Markov chain theory.

To explain it, let’s take $ P $ to be a fixed stochastic matrix.

Two states $ x $ and $ y $ are said to **communicate** with each other if
there exist positive integers $ j $ and $ k $ such that

$$
P^j(x, y) > 0
\quad \text{and} \quad
P^k(y, x) > 0
$$

In view of our discussion [above](markov_chains_I.ipynb#finite-mc-mstp), this means precisely
that

- state $ x $ can eventually be reached from state $ y $, and  
- state $ y $ can eventually be reached from state $ x $  


The stochastic matrix $ P $ is called **irreducible** if all states communicate;
that is, if $ x $ and $ y $ communicate for all $ (x, y) $ in $ S \times S $.

For example, consider the following transition probabilities for wealth of a
fictitious set of households

```ipython3
dot = Digraph(comment='Graph')
dot.attr(rankdir='LR')
dot.node("poor")
dot.node("middle class")
dot.node("rich")

dot.edge("poor", "poor", label="0.9")
dot.edge("poor", "middle class", label="0.1")
dot.edge("middle class", "poor", label="0.4")
dot.edge("middle class", "middle class", label="0.4")
dot.edge("middle class", "rich", label="0.2")
dot.edge("rich", "poor", label="0.1")
dot.edge("rich", "middle class", label="0.1")
dot.edge("rich", "rich", label="0.8")

dot
```


We can translate this into a stochastic matrix, putting zeros where
there’s no edge between nodes

$$
P :=
\left(
  \begin{array}{ccc}
     0.9 & 0.1 & 0 \\
     0.4 & 0.4 & 0.2 \\
     0.1 & 0.1 & 0.8
  \end{array}
\right)
$$

It’s clear from the graph that this stochastic matrix is irreducible: we can  eventually
reach any state from any other state.

We can also test this using [QuantEcon.py](http://quantecon.org/quantecon-py)’s MarkovChain class

```ipython3
P = [[0.9, 0.1, 0.0],
     [0.4, 0.4, 0.2],
     [0.1, 0.1, 0.8]]

mc = qe.MarkovChain(P, ('poor', 'middle', 'rich'))
mc.is_irreducible
```


Here’s a more pessimistic scenario in which  poor people remain poor forever

```ipython3
dot = Digraph(comment='Graph')
dot.attr(rankdir='LR')
dot.node("poor")
dot.node("middle class")
dot.node("rich")

dot.edge("poor", "poor", label="1.0")
dot.edge("middle class", "poor", label="0.1")
dot.edge("middle class", "middle class", label="0.8")
dot.edge("middle class", "rich", label="0.1")
dot.edge("rich", "middle class", label="0.2")
dot.edge("rich", "rich", label="0.8")

dot
```


This stochastic matrix is not irreducible since, for example, rich is not
accessible from poor.

Let’s confirm this

```ipython3
P = [[1.0, 0.0, 0.0],
     [0.1, 0.8, 0.1],
     [0.0, 0.2, 0.8]]

mc = qe.MarkovChain(P, ('poor', 'middle', 'rich'))
mc.is_irreducible
```


It might be clear to you already that irreducibility is going to be important
in terms of long run outcomes.

For example, poverty is a life sentence in the second graph but not the first.

We’ll come back to this a bit later.

### Irreducibility and Stationarity

We discussed uniqueness of the stationary in the [previous lecture](markov_chains_I.ipynb#stationary) requires the transition matrix to be everywhere positive.

In fact irreducibility is enough for the uniqueness of the stationary distribution to hold if the distribution exists.

We can revise the [theorem](markov_chains_I.ipynb#strict-stationary) into the following fundamental theorem:

### 

If $ P $ is irreducible, then $ P $ has exactly one stationary
distribution.

For proof, see Chapter 4 of [[SS23](zreferences.ipynb#id3)] or
Theorem 5.2 of [[Haggstrom02](zreferences.ipynb#id134)].


<a id='ergodicity'></a>

## Ergodicity

Under irreducibility, yet another important result obtains:

## 

If $ P $ is irreducible and $ \psi^* $ is the unique stationary
distribution, then, for all $ x \in S $,


<a id='equation-llnfmc0'></a>
$$
\frac{1}{m} \sum_{t = 1}^m \mathbf{1}\{X_t = x\}  \to \psi^*(x)
    \quad \text{as } m \to \infty \tag{22.1}
$$

Here

- $ \{X_t\} $ is a Markov chain with stochastic matrix $ P $ and initial
  distribution $ \psi_0 $  
- $ \mathbf{1}\{X_t = x\} = 1 $ if $ X_t = x $ and zero otherwise  


The result in [(22.1)](#equation-llnfmc0) is sometimes called **ergodicity**.

The theorem tells us that the fraction of time the chain spends at state $ x $
converges to $ \psi^*(x) $ as time goes to infinity.


<a id='new-interp-sd'></a>
This gives us another way to interpret the stationary distribution (provided irreducibility holds).

Importantly, the result is valid for any choice of $ \psi_0 $.

The theorem is related to [the law of large numbers](lln_clt.ipynb).

It tells us that, in some settings, the law of large numbers sometimes holds even when the
sequence of random variables is [not IID](lln_clt.ipynb#iid-violation).


<a id='mc-eg1-2'></a>

### Example 1

Recall our cross-sectional interpretation of the employment/unemployment model [discussed above](markov_chains_I.ipynb#mc-eg1-1).

Assume that $ \alpha \in (0,1) $ and $ \beta \in (0,1) $, so that irreducibility holds.

We saw that the stationary distribution is $ (p, 1-p) $, where

$$
p = \frac{\beta}{\alpha + \beta}
$$

In the cross-sectional interpretation, this is the fraction of people unemployed.

In view of our latest (ergodicity) result, it is also the fraction of time that a single worker can expect to spend unemployed.

Thus, in the long-run, cross-sectional averages for a population and time-series averages for a given person coincide.

This is one aspect of the concept  of ergodicity.


<a id='ergo'></a>

### Example 2

Another example is Hamilton [[Ham05](zreferences.ipynb#id166)] dynamics [discussed before](markov_chains_I.ipynb#mc-eg2).

The diagram of the Markov chain shows that it is **irreducible**.

Therefore, we can see the sample path averages for each state (the fraction of time spent in each state) converges to the stationary distribution regardless of the starting state

```ipython3
P = np.array([[0.971, 0.029, 0.000],
              [0.145, 0.778, 0.077],
              [0.000, 0.508, 0.492]])
ts_length = 10_000
mc = qe.MarkovChain(P)
n = len(P)
fig, axes = plt.subplots(nrows=1, ncols=n)
ψ_star = mc.stationary_distributions[0]
plt.subplots_adjust(wspace=0.35)

for i in range(n):
    axes[i].grid()
    axes[i].axhline(ψ_star[i], linestyle='dashed', lw=2, color = 'black', 
                    label = fr'$\psi^*({i})$')
    axes[i].set_xlabel('t')
    axes[i].set_ylabel(f'fraction of time spent at {i}')

    # Compute the fraction of time spent, starting from different x_0s
    for x0, col in ((0, 'blue'), (1, 'green'), (2, 'red')):
        # Generate time series that starts at different x0
        X = mc.simulate(ts_length, init=x0)
        X_bar = (X == i).cumsum() / (1 + np.arange(ts_length, dtype=float))
        axes[i].plot(X_bar, color=col, label=f'$x_0 = \, {x0} $')
    axes[i].legend()
plt.show()
```


### Example 3

Let’s look at one more example with six states [discussed before](markov_chains_I.ipynb#mc-eg3).

$$

$$

P :=
\left(
\begin{array}{cccccc}
0.86 & 0.11 & 0.03 & 0.00 & 0.00 & 0.00 \
0.52 & 0.33 & 0.13 & 0.02 & 0.00 & 0.00 \
0.12 & 0.03 & 0.70 & 0.11 & 0.03 & 0.01 \
0.13 & 0.02 & 0.35 & 0.36 & 0.10 & 0.04 \
0.00 & 0.00 & 0.09 & 0.11 & 0.55 & 0.25 \
0.00 & 0.00 & 0.09 & 0.15 & 0.26 & 0.50
\end{array}
\right)
\$$  $\$

The graph for the chain shows states are densely connected indicating that it is **irreducible**.

Similar to previous examples, the sample path averages for each state converges to the stationary distribution

```ipython3
P = [[0.86, 0.11, 0.03, 0.00, 0.00, 0.00],
     [0.52, 0.33, 0.13, 0.02, 0.00, 0.00],
     [0.12, 0.03, 0.70, 0.11, 0.03, 0.01],
     [0.13, 0.02, 0.35, 0.36, 0.10, 0.04],
     [0.00, 0.00, 0.09, 0.11, 0.55, 0.25],
     [0.00, 0.00, 0.09, 0.15, 0.26, 0.50]]

ts_length = 10_000
mc = qe.MarkovChain(P)
ψ_star = mc.stationary_distributions[0]
fig, ax = plt.subplots(figsize=(9, 6))
X = mc.simulate(ts_length)
# Center the plot at 0
ax.set_ylim(-0.25, 0.25)
ax.axhline(0, linestyle='dashed', lw=2, color = 'black', alpha=0.4)


for x0 in range(6):
    # Calculate the fraction of time for each state
    X_bar = (X == x0).cumsum() / (1 + np.arange(ts_length, dtype=float))
    ax.plot(X_bar - ψ_star[x0], label=f'$X = {x0+1} $')
    ax.set_xlabel('t')
    ax.set_ylabel(r'fraction of time spent in a state $- \psi^* (x)$')

ax.legend()
plt.show()
```


### Example 4

Let’s look at another example with two states: 0 and 1.

$$
P :=
\left(
  \begin{array}{cc}
     0 & 1\\
     1 & 0\\
  \end{array}
\right)
$$

The diagram of the Markov chain shows that it is **irreducible**

```ipython3
dot = Digraph(comment='Graph')
dot.attr(rankdir='LR')
dot.node("0")
dot.node("1")

dot.edge("0", "1", label="1.0", color='red')
dot.edge("1", "0", label="1.0", color='red')

dot
```


Unlike other Markov chains we have seen before, it has a periodic cycle — the state cycles between the two states in a regular way.

This is called [periodicity](https://www.randomservices.org/random/markov/Periodicity.html).

It is still irreducible, however, so ergodicity holds.

```ipython3
P = np.array([[0, 1],
              [1, 0]])
ts_length = 10_000
mc = qe.MarkovChain(P)
n = len(P)
fig, axes = plt.subplots(nrows=1, ncols=n)
ψ_star = mc.stationary_distributions[0]

for i in range(n):
    axes[i].grid()
    axes[i].set_ylim(0.45, 0.55)
    axes[i].axhline(ψ_star[i], linestyle='dashed', lw=2, color = 'black', 
                    label = fr'$\psi^*({i})$')
    axes[i].set_xlabel('t')
    axes[i].set_ylabel(f'fraction of time spent at {i}')

    # Compute the fraction of time spent, for each x
    for x0 in range(n):
        # Generate time series starting at different x_0
        X = mc.simulate(ts_length, init=x0)
        X_bar = (X == i).cumsum() / (1 + np.arange(ts_length, dtype=float))
        axes[i].plot(X_bar, label=f'$x_0 = \, {x0} $')

    axes[i].legend()
plt.show()
```


This example helps to emphasize the fact that asymptotic stationarity is about the distribution, while ergodicity is about the sample path.

The proportion of time spent in a state can converge to the stationary distribution with periodic chains.

However, the distribution at each state does not.

### Expectations of Geometric Sums

Sometimes we want to compute the mathematical expectation of a geometric sum, such as
$ \sum_t \beta^t h(X_t) $.

In view of the preceding discussion, this is

$$
\mathbb{E} 
    \left[
        \sum_{j=0}^\infty \beta^j h(X_{t+j}) \mid X_t 
        = x
    \right]
    = x + \beta (Ph)(x) + \beta^2 (P^2 h)(x) + \cdots
$$

By the [Neumann series lemma](eigen_II.ipynb#la-neumann), this sum can be calculated using

$$
I + \beta P + \beta^2 P^2 + \cdots = (I - \beta P)^{-1}
$$

## Exercises

## Exercise 22.1

Benhabib el al. [[BBL19](zreferences.ipynb#id257)] estimated that the transition matrix for social mobility as the following

$$
P:=\left(\begin{array}{cccccccc}0.222 & 0.222 & 0.215 & 0.187 & 0.081 & 0.038 & 0.029 & 0.006 \\ 0.221 & 0.22 & 0.215 & 0.188 & 0.082 & 0.039 & 0.029 & 0.006 \\ 0.207 & 0.209 & 0.21 & 0.194 & 0.09 & 0.046 & 0.036 & 0.008 \\ 0.198 & 0.201 & 0.207 & 0.198 & 0.095 & 0.052 & 0.04 & 0.009 \\ 0.175 & 0.178 & 0.197 & 0.207 & 0.11 & 0.067 & 0.054 & 0.012 \\ 0.182 & 0.184 & 0.2 & 0.205 & 0.106 & 0.062 & 0.05 & 0.011 \\ 0.123 & 0.125 & 0.166 & 0.216 & 0.141 & 0.114 & 0.094 & 0.021 \\ 0.084 & 0.084 & 0.142 & 0.228 & 0.17 & 0.143 & 0.121 & 0.028\end{array}\right)
$$

where each state 1 to 8 corresponds to a  percentile of wealth shares

$$
0-20 \%, 20-40 \%, 40-60 \%, 60-80 \%, 80-90 \%, 90-95 \%, 95-99 \%, 99-100 \%
$$

The matrix is recorded as `P` below

```python
P = [
    [0.222, 0.222, 0.215, 0.187, 0.081, 0.038, 0.029, 0.006],
    [0.221, 0.22,  0.215, 0.188, 0.082, 0.039, 0.029, 0.006],
    [0.207, 0.209, 0.21,  0.194, 0.09,  0.046, 0.036, 0.008],
    [0.198, 0.201, 0.207, 0.198, 0.095, 0.052, 0.04,  0.009],
    [0.175, 0.178, 0.197, 0.207, 0.11,  0.067, 0.054, 0.012],
    [0.182, 0.184, 0.2,   0.205, 0.106, 0.062, 0.05,  0.011],
    [0.123, 0.125, 0.166, 0.216, 0.141, 0.114, 0.094, 0.021],
    [0.084, 0.084, 0.142, 0.228, 0.17,  0.143, 0.121, 0.028]
    ]

P = np.array(P)
codes_B = ('1','2','3','4','5','6','7','8')
```


In this exercise,

1. show this process is asymptotically stationary and calculate the stationary distribution using simulations.  
1. use simulations to demonstrate ergodicity of this process.  

## Solution to[ Exercise 22.1](#mc_ex1)

1. 

Use the technique we learnt before, we can take the power of the transition matrix

```ipython3
P = [[0.222, 0.222, 0.215, 0.187, 0.081, 0.038, 0.029, 0.006],
     [0.221, 0.22,  0.215, 0.188, 0.082, 0.039, 0.029, 0.006],
     [0.207, 0.209, 0.21,  0.194, 0.09,  0.046, 0.036, 0.008],
     [0.198, 0.201, 0.207, 0.198, 0.095, 0.052, 0.04,  0.009],
     [0.175, 0.178, 0.197, 0.207, 0.11,  0.067, 0.054, 0.012],
     [0.182, 0.184, 0.2,   0.205, 0.106, 0.062, 0.05,  0.011],
     [0.123, 0.125, 0.166, 0.216, 0.141, 0.114, 0.094, 0.021],
     [0.084, 0.084, 0.142, 0.228, 0.17,  0.143, 0.121, 0.028]]

P = np.array(P)
codes_B = ('1','2','3','4','5','6','7','8')

np.linalg.matrix_power(P, 10)
```


We find again that rows of the transition matrix converge to the stationary distribution

```ipython3
mc = qe.MarkovChain(P)
ψ_star = mc.stationary_distributions[0]
ψ_star
```


1. 

```ipython3
ts_length = 1000
mc = qe.MarkovChain(P)
fig, ax = plt.subplots(figsize=(9, 6))
X = mc.simulate(ts_length)
ax.set_ylim(-0.25, 0.25)
ax.axhline(0, linestyle='dashed', lw=2, color = 'black', alpha=0.4)

for x0 in range(8):
    # Calculate the fraction of time for each worker
    X_bar = (X == x0).cumsum() / (1 + np.arange(ts_length, dtype=float))
    ax.plot(X_bar - ψ_star[x0], label=f'$X = {x0+1} $')
    ax.set_xlabel('t')
    ax.set_ylabel(r'fraction of time spent in a state $- \psi^* (x)$')

ax.legend()
plt.show()
```


Note that the fraction of time spent at each state quickly converges to the probability assigned to that state by the stationary distribution.

## Exercise 22.2

According to the discussion [above](#mc-eg1-2), if a worker’s employment dynamics obey the stochastic matrix

$$
P
= \left(
\begin{array}{cc}
    1 - \alpha & \alpha \\
    \beta & 1 - \beta
\end{array}
  \right)
$$

with $ \alpha \in (0,1) $ and $ \beta \in (0,1) $, then, in the long-run, the fraction
of time spent unemployed will be

$$
p := \frac{\beta}{\alpha + \beta}
$$

In other words, if $ \{X_t\} $ represents the Markov chain for
employment, then $ \bar X_m \to p $ as $ m \to \infty $, where

$$
\bar X_m := \frac{1}{m} \sum_{t = 1}^m \mathbf{1}\{X_t = 0\}
$$

This exercise asks you to illustrate convergence by computing
$ \bar X_m $ for large $ m $ and checking that
it is close to $ p $.

You will see that this statement is true regardless of the choice of initial
condition or the values of $ \alpha, \beta $, provided both lie in
$ (0, 1) $.

The result should be similar to the plot we plotted [here](#ergo)

## Solution to[ Exercise 22.2](#mc_ex2)

We will address this exercise graphically.

The plots show the time series of $ \bar X_m - p $ for two initial
conditions.

As $ m $ gets large, both series converge to zero.

```ipython3
α = β = 0.1
ts_length = 10000
p = β / (α + β)

P = ((1 - α,       α),               # Careful: P and p are distinct
     (    β,   1 - β))
mc = qe.MarkovChain(P)

fig, ax = plt.subplots(figsize=(9, 6))
ax.set_ylim(-0.25, 0.25)
ax.grid()
ax.hlines(0, 0, ts_length, lw=2, alpha=0.6)   # Horizonal line at zero

for x0, col in ((0, 'blue'), (1, 'green')):
    # Generate time series for worker that starts at x0
    X = mc.simulate(ts_length, init=x0)
    # Compute fraction of time spent unemployed, for each n
    X_bar = (X == 0).cumsum() / (1 + np.arange(ts_length, dtype=float))
    # Plot
    ax.fill_between(range(ts_length), np.zeros(ts_length), X_bar - p, color=col, alpha=0.1)
    ax.plot(X_bar - p, color=col, label=f'$X_0 = \, {x0} $')
    # Overlay in black--make lines clearer
    ax.plot(X_bar - p, 'k-', alpha=0.6)

ax.legend(loc='upper right')
plt.show()
```


## Exercise 22.3

In `quantecon` library, irreducibility is tested by checking whether the chain forms a [strongly connected component](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.is_strongly_connected.html).

However, another way to verify irreducibility is by checking whether $ A $ satisfies the following statement:

Assume A is an $ n \times n $ $ A $ is irreducible if and only if $ \sum_{k=0}^{n-1}A^k $ is a positive matrix.

(see more: [[Zha12](zreferences.ipynb#id256)] and [here](https://math.stackexchange.com/questions/3336616/how-to-prove-this-matrix-is-a-irreducible-matrix))

Based on this claim, write a function to test irreducibility.

## Solution to[ Exercise 22.3](#mc_ex3)

```ipython3
def is_irreducible(P):
    n = len(P)
    result = np.zeros((n, n))
    for i in range(n):
        result += np.linalg.matrix_power(P, i)
    return np.all(result > 0)
```


```ipython3
P1 = np.array([[0, 1],
               [1, 0]])
P2 = np.array([[1.0, 0.0, 0.0],
               [0.1, 0.8, 0.1],
               [0.0, 0.2, 0.8]])
P3 = np.array([[0.971, 0.029, 0.000],
               [0.145, 0.778, 0.077],
               [0.000, 0.508, 0.492]])

for P in (P1, P2, P3):
    result = lambda P: 'irreducible' if is_irreducible(P) else 'reducible'
    print(f'{P}: {result(P)}')
```
